In [171]:
import torch
import torch.nn as nn
import scipy.optimize as optimize
import data.datasets as dataSource
import torch.utils.data as tdata
import numpy as np
import cvxpy as cp

Designer:
- mins and max in dataset, split dataset in two 

Here: 
- get Mask with "important_columns"
- MLP with batchnorm and initial batchnorm (should replace need for scaler)
- train + val loop for new network
- constraints from min, max
- constraints from designer.py in magnetdesigner
- project_to_polytope with optimize.minimize (https://stackoverflow.com/questions/17009774/quadratic-program-qp-solver-that-only-depends-on-numpy-scipy)
- input optim loop without optimizer:
    - loss_fun and tuple of interest as in other pynb
    - create x with requires_grad
    - for step in steps
        - x.grad.zero_()
        - for param in network param.grad.zero_()
        - forward, loss, backward
        - x = x - lr * x.grad
        - x = project_to_poly()
        - x.requires_grad_(True)    

In [172]:
dataset_train, dataset_val = dataSource.Dipole_H_train, dataSource.Dipole_H_val
batch_size = 64
important_cols = ["aper_x", "aper_y", "aper_x_tapering", "yoke_x", "yoke_y", "w", "w_leg", "totalCurrent"]
input_mask = [True if col in important_cols else False for col in dataset_train.input_columns]
train_loader, val_loader = tdata.DataLoader(dataset_train,batch_size), tdata.DataLoader(dataset_val,batch_size)

target_index_dict = {col:i for i,col in enumerate(dataset_train.target_columns)}
target_cols = ["B0","gfr_x_1e-4","gfr_y_1e-4"]
output_mask = [True if col in target_cols else False for col in dataset_train.target_columns]


In [173]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [188]:
oid = {key : important_cols.index(key) for key in important_cols}

def continuous_preprocessing(x: torch.Tensor):
    # x.shape = (b,9) with batchDim b and ["aper_x", "aper_y", "aper_x_tapering", "yoke_x", "yoke_y", "w", "w_leg", "totalCurrent"]
    coil_width = 0.5 * x[:,oid["yoke_x"]] - 0.5 * x[:,oid["w"]] -  x[:,oid["aper_x_tapering"]] - x[:, oid["w_leg"]]
    coil_height = 0.5 *  x[:,oid["yoke_y"]] - 0.6 *  x[:,oid["aper_y"]] - x[:, oid["w_leg"]]
    aper_x_poleoverhang = (x[:,oid["w"]] - x[:,oid["aper_x"]]) / 2
    coil_area = coil_width * coil_height
    current_density = coil_area / x[:,oid["totalCurrent"]]
    
    B_real = x[:,oid["totalCurrent"]] * (2 * 1.25663706e-6  / x[:,oid["aper_y"]]) 

    total_current_max = ((10 * (75 * 0.01)) * 1e6) * coil_area
    B_max = x[:,oid["totalCurrent"]] * (2 * 1.25663706e-6  / x[:,oid["B_real"]]) 

    B_quot = B_real / B_max
    current_quot = x[:,oid["totalCurrent"]] / total_current_max

    #self.totalCurrent = self.B_real * self.aper_y / (2 * _scipy_constants.mu_0)
    #self.totalCurrentMax = self.B_max * self.aper_y / (2 * _scipy_constants.mu_0)

    #totalCurrentMax2 = ((6 * (75 * 0.01)) * 1e6) * coilArea
    #totalCurrentMax3 = ((4 * (75 * 0.01)) * 1e6) * coilArea
    #totalCurrentMax4 = ((2 * (75 * 0.01)) * 1e6) * coilArea

    stack = torch.cat([
        x,
        coil_width.unsqueeze(1),
        coil_height.unsqueeze(1),
        coil_area.unsqueeze(1),
        aper_x_poleoverhang.unsqueeze(1),
        current_density.unsqueeze(1),
        B_real.unsqueeze(1),
        B_max.unsqueeze(1),
        B_quot.unsqueeze(1),
        total_current_max.unsqueeze(1),
        current_quot.unsqueeze(1) 
        ],dim=1)
    return stack

In [175]:
class MyRegNet(nn.Module):
    def __init__(self,h_size=32):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(len(important_cols)+5, dtype=torch.double)
        self.fc1 = nn.Linear(len(important_cols)+5,h_size, dtype=torch.double)
        self.nl1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(h_size, dtype=torch.double)
        #self.do1 = nn.Dropout1d(0.5)
        #self.fc2 = nn.Linear(h_size,h_size, dtype=torch.double)
        #self.nl2 = nn.LeakyReLU()
        #self.bn2 = nn.BatchNorm1d(h_size, dtype=torch.double)
        #self.do2 = nn.Dropout1d()
        self.fc3 = nn.Linear(h_size,3, dtype=torch.double)
    def forward(self,x):
        x = continuous_preprocessing(x)
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.bn1(x)
        #x = self.do1(x)
        #x = self.fc2(x)
        #x = self.nl2(x)
        #x = self.bn2(x)
        #x = self.do2(x)
        x = self.fc3(x)
        return x

network = MyRegNet().to(device)

In [178]:
loss_mask_df = dataset_train.dataframe[["B0","gfr_x_1e-4","gfr_y_1e-4"]].mean()


In [180]:
n_epochs = 100
lr = 1e-4
optim = torch.optim.Adam(network.parameters(), lr)
loss_fun = nn.MSELoss()

for epoch in range(n_epochs):
    network.train()
    loss_list = []
    for x,y in train_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)

        loss_mask = torch.tensor(list(loss_mask_df)).repeat((x.shape[0],1)).to(device)

        loss = loss_fun(pred * loss_mask,selected_y * loss_mask)
        loss_list.append(loss.detach().item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    epoch_train_loss = torch.tensor(loss_list).mean()

    network.eval()
    loss_list = []
    for x,y in val_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)
        loss = loss_fun(pred,selected_y)
        loss_list.append(loss.detach().item())
    epoch_val_loss = torch.tensor(loss_list).mean()
    print(epoch,epoch_train_loss,epoch_val_loss,end="\r")


KeyboardInterrupt: 

In [133]:
torch.save(network.state_dict(), "network")

In [168]:

network = MyRegNet().to(device)
network.load_state_dict(torch.load("network"))

RuntimeError: Error(s) in loading state_dict for MyRegNet:
	size mismatch for fc1.weight: copying a param with shape torch.Size([16, 13]) from checkpoint, the shape in current model is torch.Size([32, 13]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for bn1.weight: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for bn1.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for bn1.running_mean: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for bn1.running_var: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([3, 16]) from che

In [134]:
A = []
b = []

def onehot(name, neg):
    out = [0] * len(important_cols)
    out[important_cols.index(name)] = -1 if neg else 1
    return out

for name in important_cols:
    # constraints for minima: x > a --> -x < -a
    A.append(onehot(name,True))
    b.append(-dataset_train.input_mins[name])
    
    # constraints for maxima: x < a
    A.append(onehot(name,False))
    b.append(dataset_train.input_maxs[name])

In [135]:
A_rel = []
b_rel = []

# "aper_x", "aper_y", "aper_x_tapering", "yoke_x", 
# "yoke_y", "w"     , "w_leg"              , "totalCurrent"]


# aper_x > aper_y --> aper_y - aper_x < 0
A_rel.append([-1, 1, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# aper_x < 5 * aper_y --> aper_x - 5 * aper_y < 0
A_rel.append([1,-5, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# w_leg < aper_x --> w_leg - aper_x < 0
A_rel.append([-1, 0, 0, 0, 0, 0, 1, 0])
b_rel.append(0)

## Max-made
# w < 6.6*w_leg --> w - 6.6w_leg < 0
A_rel.append([ 0, 0, 0, 0, 0, 1,-6.6, 0])
b_rel.append(0)

# w_leg < 0.83w --> w_leg - 0.83w < 0
A_rel.append([ 0, 0, 0, 0, 0,-0.83, 1, 0])
b_rel.append(0)

# Fix aspect ratio w/h between 1.9 and 2.1
# make coils squareish (aspect ratio of no more than 2 --> for sides a and be we have a - 2b < 0 and b - 2a < 0)
# coil width = 0.5 yoke_x - 0.5 w - aper_tape - w_leg
# coil height = 0.5 yoke_y - 0.6 aper_y - w_leg

# c_w/c_h > 1.9 --> 1.9*c_h - c_w < 0 --> 
#A_rel.append([ 0, -1.14, 1, -0.5, 0.95, 0.5, -0.9, 0])
#b_rel.append(0)

# c_w/c_h < 2.1 --> c_w - 2.1c_h < 0
#A_rel.append([ 0, 1.26, -1, 0.5, 1.05, -0.5, 1.1, 0])
#b_rel.append(0)

In [136]:
A = A + A_rel
b = b + b_rel

In [137]:
list(zip(A,b))

[([-1, 0, 0, 0, 0, 0, 0, 0], -0.025),
 ([1, 0, 0, 0, 0, 0, 0, 0], 0.3),
 ([0, -1, 0, 0, 0, 0, 0, 0], -0.025),
 ([0, 1, 0, 0, 0, 0, 0, 0], 0.3),
 ([0, 0, -1, 0, 0, 0, 0, 0], -0.0009473),
 ([0, 0, 1, 0, 0, 0, 0, 0], 0.0362358),
 ([0, 0, 0, -1, 0, 0, 0, 0], -0.1647062),
 ([0, 0, 0, 1, 0, 0, 0, 0], 4.1469032),
 ([0, 0, 0, 0, -1, 0, 0, 0], -0.0912832),
 ([0, 0, 0, 0, 1, 0, 0, 0], 2.791676),
 ([0, 0, 0, 0, 0, -1, 0, 0], -0.0439465),
 ([0, 0, 0, 0, 0, 1, 0, 0], 1.0247168),
 ([0, 0, 0, 0, 0, 0, -1, 0], -0.0128003),
 ([0, 0, 0, 0, 0, 0, 1, 0], 0.9068187),
 ([0, 0, 0, 0, 0, 0, 0, -1], -397.8873575),
 ([0, 0, 0, 0, 0, 0, 0, 1], 286478.8974095),
 ([-1, 1, 0, 0, 0, 0, 0, 0], 0),
 ([1, -5, 0, 0, 0, 0, 0, 0], 0),
 ([-1, 0, 0, 0, 0, 0, 1, 0], 0),
 ([0, 0, 0, 0, 0, 1, -6.6, 0], 0),
 ([0, 0, 0, 0, 0, -0.83, 1, 0], 0)]

In [138]:
#def project_to_poly(x, A_mat=A, b_vec = b):
#    def loss(curr_guess):
#        return np.linalg.norm(curr_guess-x)
#    
#    # We know coil_width = 2*coil_height
#    # Area = 2 coil_width^2 
#
#    # currDens = totalCurr / 2 coil_height^2
#    # 0.12 < currDens < 7.5 
#
#    # sqrt(0.12) < sqrt(totalCurr) / (sqrt(2) coil_height) --> coil_height < sqrt(totalCurr) / (sqrt(2)*sqrt(0.12))
#    # sqrt(7.5) > sqrt(totalCurr) / (sqrt(2) coil_height)) --> coil_height > sqrt(totalCurr) / (sqrt(2)*sqrt(7.5))
#    
#    cons = {'type':'ineq','fun':lambda x: b_vec - np.dot(A_mat,x)}
#    return optimize.minimize(loss, x,constraints=cons, method='SLSQP').x
#
##A_test = [[0,1],[1,0]]
##b_test = [1,1]
##project_to_poly([2,0.1],A_test,b_test)

In [139]:
def project_to_poly2(x, A_mat=A, b_vec = b):
    # Define decision variable
    p = cp.Variable(len(x))
    # Define objective function
    objective = cp.Minimize(cp.sum_squares(p - x))

    A_mat_dynamic = A_mat[:]
    b_vec_dynamic = b_vec[:]


    # Define constraints
    constraints = [np.array(A_mat) @ p <= np.array(b_vec)]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()


    # Optimal solution
    closest_point = p.value
    return closest_point

A_test = [[0,1],[1,0]]
b_test = [1,1]
project_to_poly2([0.5,1.2],A_test,b_test)

array([0.5, 1. ])

In [140]:
tuple_of_interest = (target_index_dict["B0"], target_index_dict["gfr_x_5e-3"],target_index_dict["gfr_y_5e-3"])
target = torch.zeros(1,33,dtype=torch.double).to(device)
target[0,tuple_of_interest[0]] = 0.079589
target[0,tuple_of_interest[1]] = 0.053487
target[0,tuple_of_interest[2]] = 0.01

tuple_of_interest = (target_index_dict["B0"], target_index_dict["gfr_x_1e-4"],target_index_dict["gfr_y_1e-4"])
target = torch.zeros(1,33,dtype=torch.double).to(device)
target[0,tuple_of_interest[0]] = 1.1720211
target[0,tuple_of_interest[1]] = 0.2803503
target[0,tuple_of_interest[2]] = 0.06

tuple_of_interest = (target_index_dict["B0"], target_index_dict["gfr_x_1e-6"],target_index_dict["gfr_y_1e-6"])
target = torch.zeros(1,33,dtype=torch.double).to(device)
target[0,tuple_of_interest[0]] = 0.479001
target[0,tuple_of_interest[1]] = 0.0402589
target[0,tuple_of_interest[2]] = 0.0396294




def create_loss_function(tuple_of_interest):
    mask = torch.zeros(1,33).to(device)
    for i in tuple_of_interest:
        mask[0, i] = 1
    mse = torch.nn.MSELoss()
    def instanciated_loss_function(x,y):
        return mse(x*mask,y*mask)
    return instanciated_loss_function

loss_fun = create_loss_function(tuple_of_interest)

In [141]:
target = torch.tensor([0.3188084,0.023373,0.01], device=device,dtype=torch.double)
loss_fun = torch.nn.MSELoss()


In [143]:
network.bn0.running_mean[:8]

tensor([1.8260e-01, 1.0954e-01, 1.4303e-02, 1.3615e+00, 8.0784e-01, 4.6866e-01,
        2.4425e-01, 3.6181e+04], device='cuda:0', dtype=torch.float64)

In [152]:
#magnet_params = torch.randn((1,8),dtype=torch.double).to(device).requires_grad_(True)
magnet_params = torch.tensor(network.bn0.running_mean[:8].clone().detach(),dtype=torch.double).unsqueeze(0).to(device).requires_grad_(True)
lr = 1e-2
network.eval()

n_steps = 100000

for step in range(n_steps):
    if magnet_params.grad is not None:
        magnet_params.grad.zero_()
    for layer in network.parameters():
        if layer.grad is not None:
            layer.grad.zero_()

    pred = network(magnet_params)
    loss = loss_fun(pred,target)
    print(step,loss.detach().cpu().numpy(),end="\r")
    loss.backward()
    with torch.no_grad():        
        magnet_params = magnet_params - lr * network.bn0.running_mean[:8] * magnet_params.grad
        proj_result = project_to_poly2(magnet_params.squeeze().detach().cpu().numpy())
        if (proj_result is None):
            magnet_params = torch.tensor(magnet_params).requires_grad_(True)
        else:        
            magnet_params = torch.tensor(proj_result).unsqueeze(0).to(device).requires_grad_(True)






/tmp/ipykernel_512206/2704530241.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  magnet_params = torch.tensor(network.bn0.running_mean[:8].clone().detach(),dtype=torch.double).unsqueeze(0).to(device).requires_grad_(True)


KeyboardInterrupt: 

In [153]:
magnet_params

tensor([[2.1630e-01, 2.1599e-01, 9.4845e-04, 1.3133e+00, 6.2674e-01, 4.1418e-01,
         8.6472e-02, 3.6181e+04]], device='cuda:0', dtype=torch.float64)

Params: 
- 2.6017e-01, 5.2035e-02, 3.2659e-01, 9.4730e-04, 1.6796e-01, 4.0373e-01, 7.4797e-01, 2.2925e-01, 3.9828e+02


Targets:
- B0: 1.1720211, gfr_x_5e-3: 0.2803503, gfr_y_5e-3: 0.06

In [16]:
import femmcreator
import magnetdesigner

In [25]:
print(*list(*(magnet_params.detach().cpu().numpy())))
dipole = magnetdesigner.designer.get_Dipole_geometry_input(*list(*(magnet_params.detach().cpu().numpy())))

femm = femmcreator.FEMM()
femm.Build(dipole.get_femminput)
femm.CreateFEMM('dipole_2')

0.20814004658580507 0.0991387739797193 0.1081780247420668 0.020102094088268248 1.3773266243023516 0.7987728374467699 0.5069761621596758 0.20814004658580507 36585.47174105747


In [181]:
group = dataset_train.dataframe[(dataset_train.dataframe["yoke_x"]==1.4824975) & 
                        (dataset_train.dataframe["yoke_y"]==0.783038) &
                        (dataset_train.dataframe["aper_x"]==0.3) &
                        (dataset_train.dataframe["aper_y"]==0.075)][important_cols + ["B0"]]

In [100]:
group

,aper_x,aper_y,aper_x_poleoverhang,aper_x_tapering,yoke_x,yoke_y,w,w_leg,totalCurrent,B0
49826,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,10742.958653,0.358991
49828,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,17904.931088,0.598375
49832,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,32228.875959,1.061886
49827,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,14323.944870,0.478710
49829,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,21485.917306,0.717982
49835,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,42971.834611,1.296217
49830,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,25066.903523,0.836991
49824,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,3580.986218,0.119470
49834,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,39390.848394,1.233042
49825,0.3,0.075,0.113679,0.011368,1.482498,0.783038,0.527358,0.226836,7161.972435,0.239236


In [182]:
network.eval()

def fun(r):
    inp = torch.tensor(r[important_cols],device=device,dtype=torch.double).unsqueeze(0)
    pred = network(inp).cpu().detach()[0,0].item()
    return pred

group.assign(pred=group.apply( lambda r: fun(r) ,axis=1))

/tmp/ipykernel_512206/718775392.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp = torch.tensor(r[important_cols],device=device,dtype=torch.double).unsqueeze(0)
/tmp/ipykernel_512206/718775392.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp = torch.tensor(r[important_cols],device=device,dtype=torch.double).unsqueeze(0)
/tmp/ipykernel_512206/718775392.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp = torch.tens

,aper_x,aper_y,aper_x_tapering,yoke_x,yoke_y,w,w_leg,totalCurrent,B0,pred
49826,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,10742.958653,0.358991,0.420434
49828,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,17904.931088,0.598375,0.561901
49832,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,32228.875959,1.061886,0.988934
49827,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,14323.944870,0.478710,0.491404
49829,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,21485.917306,0.717982,0.661069
49835,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,42971.834611,1.296217,1.273911
49830,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,25066.903523,0.836991,0.769663
49824,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,3580.986218,0.119470,0.137970
49834,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,39390.848394,1.233042,1.185815
49825,0.3,0.075,0.011368,1.482498,0.783038,0.527358,0.226836,7161.972435,0.239236,0.334893


In [154]:
dataset_train.dataframe.nunique()[important_cols]

aper_x                8
aper_y                8
aper_x_tapering      40
yoke_x             4983
yoke_y             4983
w                    40
w_leg               925
totalCurrent        162
dtype: int64

In [187]:
dataset_train.dataframe["maxCurrentDensity"].unique()

array([ 2., 10.,  4.,  6.])